In [ ]:
import os
import pandas as pd
#import WAPORWA modules
os.chdir(r'D:\Github\WAPORWA\modules') #change to modules path

import WA
from WA.dS_from_grace import main as calc_dS_basin
from WA.pickle_basin import pickle_in,pickle_out  

#Read pickle
pickle=glob.glob(r'E:\WA_Sheet1\Main\Niger\*.pickle')[-1]
BASIN=pickle_in(pickle)  

In [ ]:
BASIN_SHP= BASIN['geo_data']['basin']#r"D:\Data\GRACE\Niger_GRACE\Niger.shp"
OUT_CSV= os.path.join(BASIN['Dir','ts_GRACE.csv')#r"D:\Data\GRACE\Niger_GRACE\Niger.csv"
MASCON_DATA_FOLDER = BASIN['global_data']['grace']#r"D:\Data\GRACE\GSFC.glb.200301_201607_v02.4-ICE6G"
end_month=BASIN['end_month']
                            
calc_dS_basin(BASIN_SHP,MASCON_DATA_FOLDER,OUT_CSV)

In [ ]:
grace_basin_csv=OUT_CSV
ts_grace=pd.read_csv(grace_basin_csv,sep=',',index_col=0)
ts_day=pd.DataFrame(index=pd.date_range(start='2003-01-06',end='2016-07-14',freq='D'))
ts_grace_daily=pd.merge(ts_day,ts_grace, left_index=True,right_index=True, how='outer')
ts_grace_interpolate_daily=ts_grace_daily.interpolate()
ts_cumsum_dS_monthly_firstday=ts_grace_interpolate_daily.resample('MS').first()
ts_dS_monthly=ts_cumsum_dS_monthly_firstday.diff(2).shift(-1)/2
ts_dS_yearly=ts_dS_monthly.resample('A-{0}'.format(end_month)).sum()
